<a href="https://colab.research.google.com/github/tobonjose9/UBICO/blob/master/Tarea2EST3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt

file_path_test = 'sample_data/test_pruebas.csv'
file_path_train = 'sample_data/training_pruebas.csv'
df_test = pd.read_csv(file_path_test, low_memory=False)
df_train = pd.read_csv(file_path_train, low_memory=False)

df_nuevo = df_train['PUNT_GLOBAL']
df_nuevo_test = pd.DataFrame(columns=['ID'])
df_nuevo_test['ID'] = range(0, len(df_test))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import (mean_squared_error, mean_absolute_percentage_error,
                           r2_score, confusion_matrix, classification_report,
                           accuracy_score, roc_auc_score, roc_curve)

In [ ]:
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', 50)

In [ ]:
df = pd.read_csv('sample_data/test_pruebas.csv')

<ipython-input-17-786bebfae5a1>:1: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('sample_data/test_pruebas.csv')


In [ ]:
print("=== 1.1 Exploración de datos ===")
print(f"\nDimensiones del dataset: {df.shape}")
print("\nResumen estadístico:\n", df.describe())
print("\nValores nulos por columna:\n", df.isnull().sum().sort_values(ascending=False))

=== 1.1 Exploración de datos ===

Dimensiones del dataset: (27858, 77)

Resumen estadístico:
             PERIODO  ESTU_COD_RESIDE_DEPTO  ESTU_COD_RESIDE_MCPIO  \
count  27858.000000           27858.000000           27858.000000   
mean   20231.014717             134.103920           30262.111422   
std        0.120422            3223.923651           25914.864779   
min    20231.000000               0.000000               4.000000   
25%    20231.000000              11.000000           11001.000000   
50%    20231.000000              17.000000           17001.000000   
75%    20231.000000              52.000000           52356.000000   
max    20232.000000           99999.000000           99999.000000   

       INST_COD_INSTITUCION  ESTU_SNIES_PRGMACADEMICO  ESTU_PRGM_CODMUNICIPIO  \
count          27858.000000              27858.000000            27858.000000   
mean            2511.344569              51009.264592            24672.982483   
std             1605.205403              

In [ ]:
from datetime import datetime

def fix_year(date_str):
    try:
        return pd.to_datetime(date_str, format='%d/%m/%Y', errors='raise') # Explicitly raising errors
    except pd.errors.OutOfBoundsDatetime: # Catching the specific pandas error
        # Assuming the year is incorrectly formatted (e.g., 0015 instead of 2015)
        day, month, year = date_str.split('/')
        if len(year) == 4 and year.startswith('00'):
            year = '20' + year[2:]  # Replace '0015' with '2015', etc.
            return pd.to_datetime(f"{day}/{month}/{year}", format='%d/%m/%Y')
        else:
            # If the year format is unexpected, return NaT
            return pd.NaT

df['ESTU_FECHANACIMIENTO'] = df['ESTU_FECHANACIMIENTO'].apply(fix_year)
df['EDAD'] = (datetime.now() - df['ESTU_FECHANACIMIENTO']).dt.days // 365

num_cols = df.select_dtypes(include=['int64', 'float64']).columns
cat_cols = df.select_dtypes(include=['object']).columns

    # Imputar numéricos con mediana
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

    # Imputar categóricos con moda
for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

print("\nValores nulos después de imputación:\n", df.isnull().sum().sum())


Valores nulos después de imputación:
 0


#Codigo entero

In [ ]:
# ==============================================
# CONFIGURACIÓN INICIAL (Común para todos los items)
# ==============================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import (mean_squared_error, mean_absolute_percentage_error,
                           r2_score, confusion_matrix, classification_report,
                           accuracy_score, roc_auc_score, roc_curve)

sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', 50)

# Función auxiliar para preprocesamiento
def preprocess_data(df, is_test=False):
    if 'FECHA_NACIMIENTO' in df.columns:
        df['FECHA_NACIMIENTO'] = pd.to_datetime(df['FECHA_NACIMIENTO'])
        df['EDAD'] = (datetime.now() - df['FECHA_NACIMIENTO']).dt.days // 365

    num_cols = df.select_dtypes(include=['int64', 'float64']).columns
    cat_cols = df.select_dtypes(include=['object']).columns

    df[num_cols] = df[num_cols].fillna(df[num_cols].median())

    for col in cat_cols:
        df[col] = df[col].fillna(df[col].mode()[0])
        if df[col].nunique() > 10 and not is_test:
            freq = df[col].value_counts(normalize=True)
            df[col] = df[col].apply(lambda x: x if freq.get(x, 0) > 0.05 else 'Otros')

    return df

# ==============================================
# ITEM 1: REGRESIÓN (40%)
# ==============================================
print("\n" + "="*50)
print("ITEM 1: REGRESIÓN (40%)")
print("="*50 + "\n")

# Cargar datos
df = pd.read_csv('pruebas-del-saber-2025.csv')
df = preprocess_data(df)

# 1.1 Exploración de datos
print("\n" + "-"*40)
print("1.1 Exploración de datos")
print("-"*40)

print("\nDimensiones del dataset:", df.shape)
print("\nPrimeras filas:")
print(df.head(3))
print("\nResumen estadístico:")
print(df.describe())
print("\nValores nulos por columna:")
print(df.isnull().sum())

# 1.2 Análisis de correlación
print("\n" + "-"*40)
print("1.2 Análisis de correlación")
print("-"*40)

df_dummies = pd.get_dummies(df, drop_first=True)
corr = df_dummies.corr()['PUNT_GLOBAL'].sort_values(ascending=False)

print("\nVariables con mayor correlación positiva:")
print(corr[corr > 0.1].head(5))
print("\nVariables con mayor correlación negativa:")
print(corr[corr < -0.1].tail(5))

plt.figure(figsize=(10, 8))
sns.heatmap(df_dummies.corr()[['PUNT_GLOBAL']].sort_values('PUNT_GLOBAL', ascending=False),
            annot=True, cmap='coolwarm')
plt.title('Correlación con PUNT_GLOBAL')
plt.show()

# 2. División y modelado
print("\n" + "-"*40)
print("2. Modelado de Regresión")
print("-"*40)

X = df.drop(['PUNT_GLOBAL', 'FECHA_NACIMIENTO'], axis=1)
y = df['PUNT_GLOBAL']

num_cols = X.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())]), num_cols),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))]), cat_cols)])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())])

model.fit(X_train, y_train)

# 2.3 Evaluación
print("\nEvaluación del modelo:")
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

print(f"\nR² Train: {r2_score(y_train, train_pred):.4f}")
print(f"R² Test: {r2_score(y_test, test_pred):.4f}")
print(f"MSE Train: {mean_squared_error(y_train, train_pred):.4f}")
print(f"MSE Test: {mean_squared_error(y_test, test_pred):.4f}")
print(f"MAPE Train: {mean_absolute_percentage_error(y_train, train_pred):.4f}")
print(f"MAPE Test: {mean_absolute_percentage_error(y_test, test_pred):.4f}")

# 4. Kaggle Submission
try:
    test_kaggle = pd.read_csv('test_pruebas-del-saber-2025.csv')
    test_kaggle = preprocess_data(test_kaggle, is_test=True)

    X_kaggle = test_kaggle.drop(['FECHA_NACIMIENTO'], axis=1, errors='ignore')
    predictions = model.predict(X_kaggle)

    submission = pd.DataFrame({
        'ID': test_kaggle['ID'],
        'PUNT_GLOBAL': predictions
    })
    submission.to_csv('submission_Regression.csv', index=False)
    print("\nArchivo submission_Regression.csv creado exitosamente")
except Exception as e:
    print(f"\nError al crear submission: {str(e)}")

# 5. Hallazgos
print("\n" + "-"*40)
print("5. Principales Hallazgos")
print("-"*40)

print("\nLas variables más importantes para predecir el puntaje son:")
print(corr.head(5).index.tolist())
print("\nEl modelo explica aproximadamente el",
      round(r2_score(y_test, test_pred)*100, 2),
      "% de la variabilidad en los puntajes")
print("El error promedio (MAPE) es del",
      round(mean_absolute_percentage_error(y_test, test_pred)*100, 2),
      "%")

# ==============================================
# ITEM 2: MODELO KNN (20%)
# ==============================================
print("\n" + "="*50)
print("ITEM 2: MODELO KNN (20%)")
print("="*50 + "\n")

# 1. Prueba con diferentes K
print("\n" + "-"*40)
print("1. Prueba con diferentes valores de K")
print("-"*40)

results = []
for k in [5, 10, 20, 30]:
    model_knn = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', KNeighborsRegressor(n_neighbors=k))])

    model_knn.fit(X_train, y_train)

    train_pred = model_knn.predict(X_train)
    test_pred = model_knn.predict(X_test)

    results.append({
        'vecinos': k,
        'MSE_train': mean_squared_error(y_train, train_pred),
        'MAPE_train': mean_absolute_percentage_error(y_train, train_pred),
        'MSE_test': mean_squared_error(y_test, test_pred),
        'MAPE_test': mean_absolute_percentage_error(y_test, test_pred)
    })

results_df = pd.DataFrame(results)
print("\nResultados para diferentes valores de K:")
print(results_df.to_string(index=False))

# Selección del mejor K
best_k = results_df.loc[results_df['MSE_test'].idxmin(), 'vecinos']
print(f"\nMejor K según MSE de test: {best_k}")

# 2. Comparación con regresión
print("\n" + "-"*40)
print("2. Comparación con modelo de regresión")
print("-"*40)

print("\nComparación de métricas en test:")
print(f"Regresión Lineal - MSE: {mean_squared_error(y_test, model.predict(X_test)):.2f}")
print(f"KNN (k={best_k}) - MSE: {results_df.loc[results_df['vecinos']==best_k, 'MSE_test'].values[0]:.2f}")

if mean_squared_error(y_test, model.predict(X_test)) < results_df.loc[results_df['vecinos']==best_k, 'MSE_test'].values[0]:
    print("\nEl modelo de regresión lineal tuvo mejor desempeño en este caso")
else:
    print("\nEl modelo KNN tuvo mejor desempeño en este caso")

# Kaggle Submission
try:
    model_knn = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', KNeighborsRegressor(n_neighbors=best_k))])

    model_knn.fit(X_train, y_train)

    test_kaggle = pd.read_csv('test_pruebas-del-saber-2025.csv')
    test_kaggle = preprocess_data(test_kaggle, is_test=True)

    X_kaggle = test_kaggle.drop(['FECHA_NACIMIENTO'], axis=1, errors='ignore')
    predictions = model_knn.predict(X_kaggle)

    submission = pd.DataFrame({
        'ID': test_kaggle['ID'],
        'PUNT_GLOBAL': predictions
    })
    submission.to_csv(f'submission_Knn_k{best_k}.csv', index=False)
    print(f"\nArchivo submission_Knn_k{best_k}.csv creado exitosamente")
except Exception as e:
    print(f"\nError al crear submission: {str(e)}")

# ==============================================
# ITEM 3: MODELO GBM (20%)
# ==============================================
print("\n" + "="*50)
print("ITEM 3: MODELO GBM (20%)")
print("="*50 + "\n")

# Entrenamiento del modelo
print("\n" + "-"*40)
print("Entrenamiento y evaluación del modelo GBM")
print("-"*40)

model_gbm = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(
        n_estimators=100, learning_rate=0.1,
        max_depth=3, random_state=42))])

model_gbm.fit(X_train, y_train)

# Evaluación
train_pred = model_gbm.predict(X_train)
test_pred = model_gbm.predict(X_test)

print("\nMétricas del modelo GBM:")
print(f"R² Train: {r2_score(y_train, train_pred):.4f}")
print(f"R² Test: {r2_score(y_test, test_pred):.4f}")
print(f"MSE Train: {mean_squared_error(y_train, train_pred):.4f}")
print(f"MSE Test: {mean_squared_error(y_test, test_pred):.4f}")
print(f"MAPE Train: {mean_absolute_percentage_error(y_train, train_pred):.4f}")
print(f"MAPE Test: {mean_absolute_percentage_error(y_test, test_pred):.4f}")

# Importancia de variables
print("\n" + "-"*40)
print("Importancia de variables en el modelo GBM")
print("-"*40)

feature_names = (preprocessor.named_transformers_['cat']
               .named_steps['onehot'].get_feature_names_out(cat_cols))
all_features = list(num_cols) + list(feature_names)
importances = model_gbm.named_steps['regressor'].feature_importances_

importance_df = pd.DataFrame({
    'Feature': all_features,
    'Importance': importances
}).sort_values('Importance', ascending=False)

print("\nTop 10 variables más importantes:")
print(importance_df.head(10).to_string(index=False))

# Kaggle Submission
try:
    test_kaggle = pd.read_csv('test_pruebas-del-saber-2025.csv')
    test_kaggle = preprocess_data(test_kaggle, is_test=True)

    X_kaggle = test_kaggle.drop(['FECHA_NACIMIENTO'], axis=1, errors='ignore')
    predictions = model_gbm.predict(X_kaggle)

    submission = pd.DataFrame({
        'ID': test_kaggle['ID'],
        'PUNT_GLOBAL': predictions
    })
    submission.to_csv('submission_GBM.csv', index=False)
    print("\nArchivo submission_GBM.csv creado exitosamente")
except Exception as e:
    print(f"\nError al crear submission: {str(e)}")

# ==============================================
# ITEM 4: REGRESIÓN LOGÍSTICA (20%)
# ==============================================
print("\n" + "="*50)
print("ITEM 4: REGRESIÓN LOGÍSTICA (20%)")
print("="*50 + "\n")

# Preparación de datos
print("\n" + "-"*40)
print("Preparación de datos para regresión logística")
print("-"*40)

df['Y'] = (df['PUNT_GLOBAL'] > 172).astype(int)
print("\nDistribución de la variable objetivo Y:")
print(df['Y'].value_counts(normalize=True))

X = df.drop(['PUNT_GLOBAL', 'FECHA_NACIMIENTO', 'Y'], axis=1)
y = df['Y']

# División de datos
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# Entrenamiento del modelo
print("\n" + "-"*40)
print("Entrenamiento del modelo de regresión logística")
print("-"*40)

model_logit = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))])

model_logit.fit(X_train, y_train)

# Evaluación en training
print("\n" + "-"*40)
print("Evaluación en conjunto de training")
print("-"*40)

train_pred = model_logit.predict(X_train)
train_proba = model_logit.predict_proba(X_train)[:, 1]

print(classification_report(y_train, train_pred))
print(f"Accuracy: {accuracy_score(y_train, train_pred):.4f}")
print(f"AUC-ROC: {roc_auc_score(y_train, train_proba):.4f}")

# Matriz de confusión training
print("\nMatriz de confusión (Training):")
print(confusion_matrix(y_train, train_pred))

# Evaluación en validación
print("\n" + "-"*40)
print("Evaluación en conjunto de validación")
print("-"*40)

val_pred = model_logit.predict(X_val)
val_proba = model_logit.predict_proba(X_val)[:, 1]

print(classification_report(y_val, val_pred))
print(f"Accuracy: {accuracy_score(y_val, val_pred):.4f}")
print(f"AUC-ROC: {roc_auc_score(y_val, val_proba):.4f}")

# Matriz de confusión validación
print("\nMatriz de confusión (Validación):")
print(confusion_matrix(y_val, val_pred))

# Variables importantes
print("\n" + "-"*40)
print("Variables más importantes en el modelo")
print("-"*40)

try:
    coefficients = model_logit.named_steps['classifier'].coef_[0]
    coef_df = pd.DataFrame({
        'Feature': all_features,
        'Coefficient': coefficients
    }).sort_values('Coefficient', ascending=False)

    print("\nTop 10 variables con mayor efecto positivo:")
    print(coef_df.head(10).to_string(index=False))
    print("\nTop 10 variables con mayor efecto negativo:")
    print(coef_df.tail(10).sort_values('Coefficient').to_string(index=False))
except Exception as e:
    print(f"\nNo se pudo obtener importancia de variables: {str(e)}")